# KPIs

In [1]:
import yfinance as yf
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib as plt

In [2]:
#Retrieving Data
start = dt.date.today() - dt.timedelta(1825)
end = dt.date.today()

ohlcv = yf.download("CBA.ax", start, end)

ohlcv["Adj Close"].plot()

[*********************100%***********************]  1 of 1 completed


<AxesSubplot:xlabel='Date'>

### CAGR

In [3]:
def CAGR(DF):
    df = DF.copy()
    df["daily return"] = DF["Adj Close"].pct_change()
    df["cumulitive return"] = (1 + df["daily return"]).cumprod() 
    n = len(df)/252 
    CAGR = (df["cumulitive return"][-1])**(1/n) - 1 
    return CAGR

CAGR(ohlcv)

0.12974453901205463

### Volatility

In [4]:
def volatility(DF):
    df = DF.copy()
    df["daily return"] = DF["Adj Close"].pct_change()
    vol = df["daily return"].std() * np.sqrt(252) 
    return vol

volatility(ohlcv)

0.22877390722102647

### Sharpe Ratio

In [5]:
def sharpe(DF, rf):
    df = DF.copy()
    sharpe = (CAGR(df) - rf)/volatility(df)
    return sharpe

sharpe(ohlcv, 0.02)

0.47970741220075674

# Backtesting

In [6]:
#Retriving Monthly Data
start = dt.date.today()-dt.timedelta(425)
end = dt.date.today()-dt.timedelta(30)

asx_data = pd.read_html("https://en.wikipedia.org/wiki/S%26P/ASX_200", header=0)
asx_table = asx_data[0]

#Picking the top 30 stocks in ASX200 by market cap
asx_top30 = asx_table.sort_values(by=["Market Capitalisation"], ascending=False)[:30] 
tickers = asx_top30["Code"]

tickers

40     CBA
28     BHP
57     CSL
188    WBC
13     ANZ
73     FMG
112    NAB
109    MQG
76     GMG
194    WOW
190    WES
179    TLS
150    RIO
195    WPL
178    TCL
16     APT
6      ALL
54     COL
159    SCG
156    S32
114    NCM
174    SUN
144    QBE
176    SYD
38     BXB
74     FPH
53     COH
21     ASX
172    STO
149    RHC
Name: Code, dtype: object

In [7]:
#Creating list of top 30 stocks and their monthly returns
stock_list = {} 
monthly_return_df = pd.DataFrame()

for item in tickers:
    stock_list[item] = yf.download(item+".ax", start, end, interval="1mo")
    stock_list[item].dropna(inplace=True)
    stock_list[item]["gain"] = stock_list[item]["Adj Close"].pct_change()
    monthly_return_df[item] = stock_list[item]["gain"]
    monthly_return_df.dropna(inplace=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [8]:
monthly_return_df

CBA       BHP       CSL       WBC       ANZ       FMG  \
Date                                                                     
2020-06-01  0.088941  0.034065  0.039027  0.042393  0.041923 -0.003597   
2020-07-01  0.025641  0.025963 -0.058885 -0.047911 -0.036481  0.257040   
2020-08-01 -0.041011  0.031565  0.058904  0.026331  0.017817  0.000574   
2020-09-01 -0.055259 -0.060934  0.003461 -0.039909 -0.044997 -0.011932   
2020-10-01  0.085049 -0.032155  0.007219  0.063539  0.092334  0.065644   
2020-11-01  0.145610  0.126998  0.034149  0.123953  0.203615  0.049511   
2020-12-01  0.038447  0.114526 -0.047750 -0.021516  0.020880  0.285244   
2021-01-01  0.017050  0.026632 -0.040469  0.090862  0.044493 -0.069996   
2021-02-01 -0.023351  0.127870 -0.033601  0.127307  0.103754  0.106471   
2021-03-01  0.074336 -0.077956  0.007845  0.024769  0.076806 -0.170884   
2021-04-01  0.034146  0.080843  0.029802  0.023351  0.019872  0.203439   
2021-05-01  0.119946  0.003145  0.070254  0.057646 -0.001044 -0.007083   

                 NAB       MQG       GMG       WOW  ...       NCM       SUN  \
Date                                                ...                       
2020-06-01  0.041443  0.097272 -0.033832  0.054895  ...  0.031066  0.000000   
2020-07-01 -0.030735  0.041231  0.151370  0.037554  ...  0.115763 -0.073673   
2020-08-01  0.015289  0.034092  0.081512  0.030765  ... -0.091529  0.087719   
2020-09-01 -0.010039 -0.063665 -0.020208 -0.086280  ... -0.015312 -0.081864   
2020-10-01  0.047887  0.060049  0.025641  0.060253  ... -0.064020 -0.028402   
2020-11-01  0.230645  0.094043  0.015217 -0.030922  ... -0.079001  0.228989   
2020-12-01  0.001095  0.008272  0.012313  0.063007  ... -0.042703 -0.034688   
2021-01-01  0.041593 -0.051127 -0.057722  0.039430  ... -0.025601  0.034908   
2021-02-01  0.046729  0.084323 -0.063348 -0.035732  ... -0.019108 -0.013889   
2021-03-01  0.055195  0.072642  0.094203  0.037310  ... -0.001064  0.021215   
2021-04-01  0.025385  0.050121  0.043598 -0.025443  ...  0.085995  0.061616   
2021-05-01  0.010878 -0.053337  0.027499  0.059033  ...  0.067873  0.058040   

                 QBE       SYD       BXB       FPH       COH       ASX  \
Date                                                                     
2020-06-01  0.003398 -0.030769 -0.066151  0.185132 -0.021139 -0.031753   
2020-07-01  0.118510 -0.077601 -0.008280  0.007004  0.006828 -0.031975   
2020-08-01  0.070636  0.121778  0.028757  0.018740  0.010146  0.056745   
2020-09-01 -0.185490  0.024476 -0.054103 -0.088849  0.030445 -0.072361   
2020-10-01 -0.041812 -0.069966 -0.075086  0.072971  0.071616 -0.002937   
2020-11-01  0.212121  0.231193  0.142857  0.023483  0.038835 -0.031646   
2020-12-01 -0.147000 -0.044709 -0.032847 -0.077771 -0.142546 -0.066269   
2021-01-01 -0.058617 -0.107644 -0.002830  0.048421  0.044656 -0.002917   
2021-02-01  0.156912  0.024476 -0.063387 -0.160681  0.070756 -0.058365   
2021-03-01  0.035522  0.056314  0.068687  0.084102 -0.002081  0.051035   
2021-04-01  0.024948  0.000000 -0.003959  0.144267  0.060375  0.044274   
2021-05-01  0.110548 -0.051696  0.040385 -0.169492  0.020404  0.044944   

                 STO       RHC  
Date                            
2020-06-01 -0.011194 -0.050393  
2020-07-01 -0.005660 -0.068250  
2020-08-01  0.075901  0.049693  
2020-09-01 -0.134908  0.015678  
2020-10-01 -0.030738 -0.056447  
2020-11-01  0.302326  0.011227  
2020-12-01  0.017857 -0.013799  
2021-01-01  0.038278  0.012705  
2021-02-01  0.110599  0.048118  
2021-03-01 -0.008705  0.015151  
2021-04-01 -0.016901  0.012421  
2021-05-01 -0.030086 -0.056744  

[12 rows x 30 columns]

In [9]:
#Creating/Backtesting Strategy
def strategy(DF, m, x):
    df = DF.copy()  
    x = 3 #no. of stocks to replace
    m = 6 #no. of stocks in portfolio each month
    portfolio = monthly_return_df.iloc[0].sort_values(ascending = False)[:m].index.tolist() #selecing the top 6 stocks by returns
    monthly_return_portfolio = [0] 
    monthly_stocks = pd.DataFrame() 
    for i in range(len(df)):
        monthly_return_portfolio.append(df[portfolio].iloc[i,:].mean())
        bad_stocks = df[portfolio].iloc[i,:].sort_values(ascending=True)[:x]
        portfolio = [t for t in portfolio if t not in bad_stocks]
        fill = m - len(portfolio)
        new_stocks = df.iloc[i,:].sort_values(ascending=False)[:fill].index.tolist()
        portfolio = portfolio + new_stocks
        print(portfolio)
    strategy_return_df = pd.DataFrame(np.array(monthly_return_portfolio), columns=["monthly_return_portfolio"])
    strategy_return_df.drop(0, inplace=True) 
    return strategy_return_df

strategy_return_df = strategy(monthly_return_df, 6, 3)
strategy_return_df

['APT', 'FPH', 'COL', 'APT', 'FPH', 'COL']
['APT', 'APT', 'FMG', 'GMG', 'APT', 'QBE']
['APT', 'APT', 'APT', 'APT', 'SYD', 'SCG']
['SYD', 'SCG', 'ALL', 'TCL', 'COH', 'SYD']
['SCG', 'ALL', 'COH', 'APT', 'ANZ', 'CBA']
['SCG', 'ANZ', 'CBA', 'SCG', 'STO', 'WPL']
['ANZ', 'CBA', 'STO', 'FMG', 'APT', 'RIO']
['ANZ', 'STO', 'APT', 'APT', 'WBC', 'WES']
['ANZ', 'STO', 'WBC', 'QBE', 'RIO', 'BHP']
['ANZ', 'WBC', 'QBE', 'ALL', 'TLS', 'GMG']
['QBE', 'ALL', 'GMG', 'FMG', 'APT', 'FPH']
['QBE', 'ALL', 'GMG', 'CBA', 'QBE', 'ALL']


monthly_return_portfolio
1                   0.147683
2                   0.063012
3                   0.192509
4                  -0.083825
5                  -0.034091
6                   0.137304
7                   0.013240
8                   0.023970
9                   0.002114
10                 -0.013137
11                  0.031821
12                 -0.024255

In [16]:
#Adjusting CAGR for monthly data
def CAGR(DF): 
    df = DF.copy()
    df["cumulitive return"] = (1 + df).cumprod()
    n = len(df)/12
    CAGR = (df["cumulitive return"].tolist()[-1])**(1/n) - 1
    return CAGR

CAGR(strategy_return_df['monthly_return_portfolio'])

0.46521589404564323

### Comparison to buying and holding all 30 stocks with equal weight (Benchmark)

In [11]:
monthly_return_df['monthly_return_portfolio'] = monthly_return_df.mean(axis=1)
monthly_return_df['monthly_return_portfolio']

Date
2020-06-01    0.030559
2020-07-01    0.018949
2020-08-01    0.033424
2020-09-01   -0.041598
2020-10-01    0.011467
2020-11-01    0.111145
2020-12-01    0.008000
2021-01-01    0.011387
2021-02-01    0.013269
2021-03-01    0.022552
2021-04-01    0.043324
2021-05-01    0.009368
Name: monthly_return_portfolio, dtype: float64

In [12]:
strategy_return_df['benchmark_return'] = monthly_return_df['monthly_return_portfolio'].to_numpy()
strategy_return_df['rP - rB'] = strategy_return_df['benchmark_return'] - strategy_return_df['monthly_return_portfolio']
strategy_return_df

monthly_return_portfolio  benchmark_return   rP - rB
1                   0.147683          0.030559 -0.117124
2                   0.063012          0.018949 -0.044063
3                   0.192509          0.033424 -0.159086
4                  -0.083825         -0.041598  0.042227
5                  -0.034091          0.011467  0.045559
6                   0.137304          0.111145 -0.026158
7                   0.013240          0.008000 -0.005240
8                   0.023970          0.011387 -0.012583
9                   0.002114          0.013269  0.011155
10                 -0.013137          0.022552  0.035689
11                  0.031821          0.043324  0.011504
12                 -0.024255          0.009368  0.033624

In [13]:
def CAGR(DF): 
    df = DF.copy()
    df["cumulitive return"] = (1 + df).cumprod()
    n = len(df)/12
    CAGR = (df["cumulitive return"].tolist()[-1])**(1/n) - 1
    return CAGR

def volatility(DF):
    df = DF.copy()
    vol = df.std() * np.sqrt(12) 
    return vol

def sharpe(DF, rf):
    df = DF.copy()
    sharpe = (CAGR(df) - rf)/volatility(df)
    return sharpe

In [14]:
IR = (CAGR(strategy_return_df['monthly_return_portfolio'])-CAGR(strategy_return_df['benchmark_return']))/volatility(strategy_return_df['rP - rB'])

In [15]:
print('Benchmark CAGR:', CAGR(strategy_return_df['benchmark_return']))
print('Benchmark Volatility:', volatility(strategy_return_df['benchmark_return']))
print('Benchmark Sharpe:', sharpe(strategy_return_df['benchmark_return'], 0.02))
print('-----------------------------------------')
print('Active CAGR:', CAGR(strategy_return_df['monthly_return_portfolio']))
print('Active Volatility:', volatility(strategy_return_df['monthly_return_portfolio']))
print('Active Sharpe:', sharpe(strategy_return_df['monthly_return_portfolio'], 0.02))
print('-----------------------------------------')
print('Information Ratio:', (CAGR(strategy_return_df['monthly_return_portfolio'])-CAGR(strategy_return_df['benchmark_return']))\
      /volatility(strategy_return_df['rP - rB']))

Benchmark CAGR: 0.27426390704932446
Benchmark Volatility: 0.12049853758353615
Benchmark Sharpe: 2.110099526088064
-----------------------------------------
Active CAGR: 0.46521589404564323
Active Volatility: 0.2861444525011196
Active Sharpe: 1.5559130717164655
-----------------------------------------
Information Ratio: 0.8568867223250769
